# ایزور اے آئی ایجنٹس کے ساتھ ماڈل کانٹیکسٹ پروٹوکول (MCP) سپورٹ - پائتھون

یہ نوٹ بک دکھاتی ہے کہ ایزور اے آئی ایجنٹس کو ماڈل کانٹیکسٹ پروٹوکول (MCP) ٹولز کے ساتھ پائتھون میں کیسے استعمال کیا جا سکتا ہے۔ یہ بتاتی ہے کہ ایک ذہین ایجنٹ کیسے بنایا جا سکتا ہے جو بیرونی MCP سرورز (جیسے مائیکروسافٹ لرن) کو بغیر کسی کلیدی تصدیق کے بہتر صلاحیتوں کے لیے استعمال کر سکے۔


## مطلوبہ پائتھون پیکجز انسٹال کریں

سب سے پہلے، ہمیں ضروری پائتھون پیکجز انسٹال کرنے ہوں گے:
- **azure-ai-projects**: کور ایذور اے آئی پروجیکٹس ایس ڈی کے  
- **azure-ai-agents**: ایذور اے آئی ایجنٹس ایس ڈی کے، ایجنٹس بنانے اور مینج کرنے کے لیے  
- **azure-identity**: ڈیفالٹ ایذور کریڈینشل کے ذریعے بغیر کلید کے تصدیق فراہم کرتا ہے  
- **mcp**: پائتھون کے لیے ماڈل کانٹیکسٹ پروٹوکول کا نفاذ  


## کلیدی تصدیق کے فوائد

یہ نوٹ بک **کلیدی تصدیق** کے فوائد کو ظاہر کرتی ہے، جو کئی اہم فوائد فراہم کرتی ہے:
- ✅ **کوئی API کیز کا انتظام نہیں** - Azure کی شناخت پر مبنی تصدیق استعمال کرتا ہے
- ✅ **بہتر سیکیورٹی** - کوڈ یا کنفیگریشن فائلز میں کوئی راز محفوظ نہیں کیے جاتے
- ✅ **خودکار اسناد کی گردش** - Azure اسناد کے زندگی کے دور کا انتظام کرتا ہے
- ✅ **رول پر مبنی رسائی کنٹرول** - Azure RBAC کے ذریعے باریک بینی سے اجازتیں فراہم کرتا ہے
- ✅ **کثیر ماحول کی حمایت** - ترقی اور پیداوار کے درمیان بغیر کسی رکاوٹ کے کام کرتا ہے

`DefaultAzureCredential` خود بخود بہترین دستیاب اسناد کے ماخذ کا انتخاب کرتا ہے:
1. **مینجڈ شناخت** (جب Azure میں چل رہا ہو)
2. **Azure CLI** اسناد (مقامی ترقی کے دوران)
3. **Visual Studio** اسناد
4. **ماحولیاتی متغیرات** (اگر ترتیب دی گئی ہوں)
5. **انٹرایکٹو براؤزر** تصدیق (بطور آخری حل)


## کی لیس تصدیق کی ترتیب

**کی لیس تصدیق کے لیے ضروریات:**

### مقامی ترقی کے لیے:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure ماحول کے لیے:
- اپنے Azure وسائل پر **سسٹم-اسائنڈ مینیجڈ آئیڈینٹیٹی** کو فعال کریں
- مینیجڈ آئیڈینٹیٹی کو مناسب **RBAC رولز** تفویض کریں:
  - Azure OpenAI تک رسائی کے لیے `Cognitive Services OpenAI User`
  - Azure AI Projects تک رسائی کے لیے `AI Developer`

### ماحول کے متغیرات (اختیاری):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**کوئی API کیز یا کنکشن اسٹرنگز کی ضرورت نہیں!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## ضروری لائبریریاں درآمد کریں

ضروری پائتھون ماڈیولز درآمد کریں:
- **os, time**: ماحول کے متغیرات اور تاخیر کے لیے معیاری پائتھون لائبریریاں
- **AIProjectClient**: Azure AI پروجیکٹس کے لیے مرکزی کلائنٹ
- **DefaultAzureCredential**: Azure خدمات کے لیے بغیر کلید کے تصدیق
- **MCP سے متعلق کلاسز**: MCP ٹولز بنانے اور ان کا انتظام کرنے اور منظوریوں کو سنبھالنے کے لیے


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## ایم سی پی سرور کی ترتیبات ترتیب دیں

ماحول کی متغیرات کے ذریعے ایم سی پی سرور کی ترتیب کو سیٹ کریں، ساتھ ہی ڈیفالٹ متبادل:
- **MCP_SERVER_URL**: ایم سی پی سرور کا یو آر ایل (ڈیفالٹ Microsoft Learn API ہے)
- **MCP_SERVER_LABEL**: ایم سی پی سرور کی شناخت کے لیے ایک لیبل (ڈیفالٹ "mslearn" ہے)

یہ طریقہ مختلف ماحول میں لچکدار ترتیب کی اجازت دیتا ہے۔


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI پروجیکٹ کلائنٹ بنائیں (کی لیس تصدیق)

Azure AI پروجیکٹ کلائنٹ کو **کی لیس تصدیق** کے ذریعے شروع کریں:
- **endpoint**: Azure AI Foundry پروجیکٹ کا endpoint URL
- **credential**: محفوظ، کی لیس تصدیق کے لیے `DefaultAzureCredential()` استعمال کرتا ہے
- **کوئی API keys درکار نہیں**: خودکار طور پر بہترین دستیاب credential تلاش کرتا ہے اور استعمال کرتا ہے

**تصدیق کا طریقہ کار:**
1. Managed Identity کی جانچ کرتا ہے (Azure ماحول میں)
2. Azure CLI credentials پر واپس آتا ہے (مقامی ترقی کے لیے)
3. ضرورت کے مطابق دیگر دستیاب credential ذرائع استعمال کرتا ہے

یہ طریقہ کار آپ کے کوڈ میں API keys یا connection strings کو منظم کرنے کی ضرورت ختم کر دیتا ہے۔


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## ایم سی پی ٹول کی تعریف بنائیں

ایم سی پی ٹول بنائیں جو مائیکروسافٹ لرن ایم سی پی سرور سے جڑتا ہے:
- **server_label**: ایم سی پی سرور کے لیے شناخت کنندہ
- **server_url**: ایم سی پی سرور کا یو آر ایل اینڈ پوائنٹ
- **allowed_tools**: اختیاری فہرست جو ان ٹولز کو محدود کرتی ہے جو استعمال کیے جا سکتے ہیں (خالی فہرست تمام ٹولز کی اجازت دیتی ہے)

یہ ٹول ایجنٹ کو مائیکروسافٹ لرن کی دستاویزات اور وسائل تک رسائی فراہم کرے گا۔


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## ایجنٹ بنائیں اور گفتگو انجام دیں (کی لیس ورک فلو)

یہ جامع سیکشن مکمل **کی لیس ایجنٹ ورک فلو** کو ظاہر کرتا ہے:

1. **AI ایجنٹ بنائیں**: GPT-4.1 نینو ماڈل اور MCP ٹولز کے ساتھ ایجنٹ سیٹ اپ کریں
2. **تھریڈ بنائیں**: بات چیت کے لیے ایک گفتگو تھریڈ قائم کریں
3. **پیغام بھیجیں**: ایجنٹ سے Azure OpenAI اور OpenAI کے فرق کے بارے میں پوچھیں
4. **ٹول کی منظوری سنبھالیں**: جب ضرورت ہو تو MCP ٹول کالز کو خودکار طور پر منظور کریں
5. **عمل کی نگرانی کریں**: ایجنٹ کی پیش رفت کو ٹریک کریں اور ضروری اقدامات کریں
6. **نتائج دکھائیں**: گفتگو اور ٹول کے استعمال کی تفصیلات ظاہر کریں

**کی لیس خصوصیات:**
- ✅ **کوئی ہارڈ کوڈڈ راز نہیں** - تمام تصدیق Azure شناخت کے ذریعے کی جاتی ہے
- ✅ **پہلے سے محفوظ** - رول بیسڈ ایکسیس کنٹرول استعمال کرتا ہے
- ✅ **آسان تعیناتی** - کسی اسناد کے انتظام کی ضرورت نہیں
- ✅ **آڈٹ کے لیے موزوں** - تمام رسائی Azure شناخت کے ذریعے ٹریک کی جاتی ہے

ایجنٹ MCP ٹولز کا استعمال کرتے ہوئے Microsoft Learn وسائل تک مکمل سیکیورٹی کے ساتھ اور بغیر کسی API کلید کے انتظام کے رسائی حاصل کرے گا۔


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
